### OPERATIONS HAUT NIVEAU

In [2]:
import pandas as pd
import numpy as np
from time import time
rng = np.random.default_rng(seed=int(time()))
pd.__version__

'2.1.4'

In [3]:
url = "http://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
penguins_df = pd.read_csv(
    url,
    encoding="utf8"
)
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


#### GROUP BY

In [4]:
# poids moyens des penguins par espèce ou sexe
# => slectionnner les champs utiles
# => les champs du groupby doivent être dans le df (comme en SQL)
gb = penguins_df[["body_mass_g","species","sex"]].groupby(by=["species","sex"])
# gb["body_mass_g"].mean()
grouped_df = gb["body_mass_g"].agg(["mean", "max"])
grouped_df

mean     max
species   sex                        
Adelie    FEMALE  3368.835616  3900.0
          MALE    4043.493151  4775.0
Chinstrap FEMALE  3527.205882  4150.0
          MALE    3938.970588  4800.0
Gentoo    FEMALE  4679.741379  5200.0
          MALE    5484.836066  6300.0

In [5]:
# Notion d'index multiple: requêtage
grouped_df.loc["Adelie", "mean"]
grouped_df.loc[("Adelie", "FEMALE")]

mean    3368.835616
max     3900.000000
Name: (Adelie, FEMALE), dtype: float64

In [6]:
# manipulation
grouped_df.swaplevel(1,0, axis=0)

,,mean,max
sex,species,,
FEMALE,Adelie,3368.835616,3900.0
MALE,Adelie,4043.493151,4775.0
FEMALE,Chinstrap,3527.205882,4150.0
MALE,Chinstrap,3938.970588,4800.0
FEMALE,Gentoo,4679.741379,5200.0
MALE,Gentoo,5484.836066,6300.0


#### SEGMENTATION

In [7]:
# on veut repartir les penguins par catégories de poids i.e avec des intervalles min et max
# on veut 3 catégories arbitraires légers, medium et lourds avec des seuils linéaires i.e longueur d'intervalle égales
# => max - min égales => |----------|----------|----------|
#                       min  léger    médium       lourd   max
penguins_df 

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [10]:
# 5 intervalles de même longueur entre 0 et 100
np.linspace(0, 100, num=5)
lin = np.linspace(np.sqrt(np.pi**2), np.sqrt(6*np.pi**2), num=5)
np.diff(lin)

array([1.13842658, 1.13842658, 1.13842658, 1.13842658])

In [28]:
# on peut unpacker les series
w_min, w_max = penguins_df["body_mass_g"].agg(["min", "max"])
# 3 catégories donc 4 seuils
intervals = np.linspace(w_min, w_max, num=4)
print(intervals)
body_mass_category = pd.cut(
    penguins_df["body_mass_g"], 
    bins=intervals,
    labels=["light", "medium", "heavy"],
    # gère si le min (resp. max) est exclu de l'intervalle
    right=False
)
penguins_df.insert(
    6,
    value=body_mass_category, 
    column="body_mass_category"
)

[2700. 3900. 5100. 6300.]


ValueError: cannot insert body_mass_category, already exists

In [34]:
penguins_df.groupby(["body_mass_category", "sex"])["species"].count()

C:\Users\Admin stagiaire.DESKTOP-8967908\AppData\Local\Temp\ipykernel_1392\87690966.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  penguins_df.groupby(["body_mass_category", "sex"])["species"].count()


body_mass_category  sex   
light               FEMALE    103
                    MALE       37
medium              FEMALE     58
                    MALE       77
heavy               FEMALE      4
                    MALE       53
Name: species, dtype: int64

In [37]:
# segmentation avec des quantiles
penguins_df["body_mass_quantile"] = pd.qcut(
    penguins_df["body_mass_g"],
    q=[0, 0.33, 0.66, 1],
    labels=["q_light", "q_medium", "q_heavy"]
)


In [38]:
penguins_df.groupby(["body_mass_quantile", "sex"])["species"].count()

C:\Users\Admin stagiaire.DESKTOP-8967908\AppData\Local\Temp\ipykernel_1392\3063208108.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  penguins_df.groupby(["body_mass_quantile", "sex"])["species"].count()


body_mass_quantile  sex   
q_light             FEMALE    93
                    MALE      20
q_medium            FEMALE    30
                    MALE      78
q_heavy             FEMALE    42
                    MALE      70
Name: species, dtype: int64

#### PIVOT AKA TABLEAU CROISE DYNAMIQUE

##### pivot
* fabrication du df:
  1. une colonne student avec 10 valeurs student_{i} répétées 4 fois 
  2. une colonne subject avec 4 valeurs ["maths", "english", "biology", "physics"]  répétées dans l'ordre 10 fois
  3. une colonne note avec des entiers entre 0 et 20 (avec les 0.5 possibles)
  4. une colonne coeff qui répètent les mêmes valeurs en fonction du subject

